In [45]:
# Simple IK for 2 DOF leg
# Author : Avadesh Meduri
# Date : 12/02/2021

import numpy as np
import pinocchio as pin
import crocoddyl

from robots import TwoDofManipulator, vis_two_dof
from action_model import DifferentialFwdKinematics

In [61]:
robot_model, data = TwoDofManipulator()
q0 = np.array([-np.pi/6.0, np.pi/4.0]).T
x0 = np.concatenate([q0, pin.utils.zero(state.nv)])
# Running and terminal action models
DT = 1e-3
T = 100

In [73]:
running_cost_arr = []
x = -1.
for t in range(T):
    # Create the cost functions
    omega = np.pi/T
    x += 2/T
    target = np.array([x , -1.5 + np.sin(omega*t), 0.0])
#     Mref = crocoddyl.FrameTranslation(robot_model.getFrameId("FOOT"), target)
#     state = crocoddyl.StateMultibody(robot_model)
#     goalTrackingCost = crocoddyl.CostModelFrameTranslation(state, Mref)
    tmp = crocoddyl.FrameMotion(robot_model.getFrameId("FOOT"), pin.Motion(np.array([10,0,0,0,0,0])))
    velTrackingCost = crocoddyl.CostModelFrameVelocity(state, tmp)
    
    xRegCost = crocoddyl.CostModelState(state)
    uRegCost = crocoddyl.CostModelControl(state)

    # Create cost model per each action model
    runningCostModel = crocoddyl.CostModelSum(state)
    terminalCostModel = crocoddyl.CostModelSum(state)

    # Then let's added the running and terminal cost functions
#     runningCostModel.addCost("gripperPose", goalTrackingCost, 1e2)
    runningCostModel.addCost("grippervel", velTrackingCost, 1e2)
    runningCostModel.addCost("stateReg", xRegCost, 1e-4)
    runningCostModel.addCost("ctrlReg", uRegCost, 1e-7)
#     terminalCostModel.addCost("gripperPose", goalTrackingCost, 1e5)
    runningCostModel.addCost("grippervel", velTrackingCost, 1e2)
    terminalCostModel.addCost("stateReg", xRegCost, 1e-4)
    terminalCostModel.addCost("ctrlReg", uRegCost, 1e-7)

    runningModel = crocoddyl.IntegratedActionModelEuler(
        DifferentialFwdKinematics(state, runningCostModel), DT)
    running_cost_arr.append(runningModel)
    
terminalModel = crocoddyl.IntegratedActionModelEuler(
    DifferentialFwdKinematics(state,terminalCostModel), 0.)

In [74]:
problem = crocoddyl.ShootingProblem(x0, running_cost_arr, terminalModel)

# Creating the DDP solver for this OC problem, defining a logger
ddp = crocoddyl.SolverDDP(problem)
log = crocoddyl.CallbackLogger()
ddp.setCallbacks([log,
                  crocoddyl.CallbackVerbose(),
                  ])
# # Solving it with the DDP algorithm
ddp.solve()


True

In [75]:
vis_two_dof(robot_model, data, np.array(ddp.xs), 1)

<source type="video/mp4" src="data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAAAjKG1kYXQAAAKuBgX//6rcRem9
5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTUyIHIyODU0IGU5YTU5MDMgLSBILjI2NC9NUEVHLTQg
QVZDIGNvZGVjIC0gQ29weWxlZnQgMjAwMy0yMDE3IC0gaHR0cDovL3d3dy52aWRlb2xhbi5vcmcv
eDI2NC5odG1sIC0gb3B0aW9uczogY2FiYWM9MSByZWY9MyBkZWJsb2NrPTE6MDowIGFuYWx5c2U9
MHgzOjB4MTEzIG1lPWhleCBzdWJtZT03IHBzeT0xIHBzeV9yZD0xLjAwOjAuMDAgbWl4ZWRfcmVm
PTEgbWVfcmFuZ2U9MTYgY2hyb21hX21lPTEgdHJlbGxpcz0xIDh4OGRjdD0xIGNxbT0wIGRlYWR6
b25lPTIxLDExIGZhc3RfcHNraXA9MSBjaHJvbWFfcXBfb2Zmc2V0PS0yIHRocmVhZHM9OSBsb29r
YWhlYWRfdGhyZWFkcz0xIHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFj
ZWQ9MCBibHVyYXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJh
bWlkPTIgYl9hZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdl
aWdodHA9MiBrZXlpbnQ9MjUwIGtleWludF9taW49MjUgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVz
aD0wIHJjX2xvb2thaGVhZD00MCByYz1jcmYgbWJ0cmVlPTEgY3JmPTIzLjAgcWNvbXA9MC42MCBx
cG1pbj0wIHFwbWF4PTY5IHFwc3RlcD00IGlwX3JhdGlvPTEuNDAgYXE9MToxLjAwAIAAAAvCZYiE
ACf//vWxfApqyfOKDOgyLuGXJMmutiLibQDAAAADAAVQU9eBiH/MBP1aFioLMocHty2rZGB3GXDn
YZWCairtzfSFne2+SYIAAAMAak3/vcE97DVfTPu8AFRj7DTC2HDCLQ4mMIPBDTspEaG+CUEz3ndb
DTRVmQJ7BpH/9Xc1CRA2cV0fH5OVm5p0KBAGAcSjw3P7W9lOA8NYYgQjvuvRvNnFryjD070yUubi
wGvPkrk6BXiXhTea0gModMEtzpRddiUFItsScMPa2RQCJ8TBcv5wHoUIwxx+dCrJFt1skHgmMf4+
LEhh+JBNFXY3JYv0EMoSChllf9tRyoWpy3Q9nbRbku7u3WWEXLNXpbkn70Hn+Gslhf2d0oxzUzc2
TGev/k0J0x8/rNJP3Dp/uXltBgGv1xAoCFpA/+TCnd/5Quc3b2KJr2q4di0d5/yrjNTKKlFheU0U
URH5J0pKaSj8jAjtHWblq7l7NVOAtv3D56QzBzspmseWqTKZY/p9rZ44e9XtvnqYz+gBumAI2qas
KbWXEshqC8VgPoiS9Xh7uX0yB4/lBD2KLQFDIokJZ2feIRBQ8qwS1ELqLROMfySwKXc23ECRRBTY
ge4LrrWCwIfBa3wyTBrlYEaPTMY/ZsNWNphujcJ0XDM3UT2va+f1UYWantFqjQbGrGJso45BYu0F
/kQGf6B6KbgUJiiLkYXy9wcXJthSLqckm5AFC5+sdzA0sVYsHvgev1JnWSLXZXAH3Qa575zMAgkY
Hsg8rKyBKA6/JTVfTP7WjG2DLCC/lIqhJ6rbL4T6mrBwwSmbjOy9g+5rffJ2V5HjypsfXhm9VPb/
uxyCaCEr5k6WMeUeHkW8EIa/7Rr0sytqN3LCawalEtdzZKTD3i4bENTkbOZkiwlcVLnsCGP8llbM
Y/433GPvSRS5HDxog6v4Z8/SEQbidMEGGBnQUQvfAplZbekY/z+hkS42SNYlitNd4HsaqKYheBx+
lkz0OhE7G5Hppphm/KkJE81SQd7VPZ7kekXBmAo3f3wshZlP/nhBQBud/YEUzSlqZ6EYTDv89PJq
o9xswL3T3AJ5raRmLlq6DJlHC58TuK4tndZyHeqhb9d+oR5AHjuIdf8W72nso8Nd7yspsproA2bj
KbGAY9oFZly8vaJ8A58CvqKMTBVJPQxtsNXQAbIH4kcS4WEqK2jM8pkVa2r3aRBfZ8VvsMkW8d0E
fSAqqSRqmqRxpx+atC3WScKT2GJupnb7k/een4+HI2OT//HL5jR8XfZoIln/bJBLuhPdM5Gqbxzt
g3/9+AfJnmWs5LUFTreDMufGsfRDzkQYHvpOzHuh918awUgHHFQoWakn+uIu1MPa79cx154p8jXA
qe6yqdcn77e028XEWAMWqnmK9CtcHZ36MP8Soz7mAMHaGT+PFp5RMEg5/H45XAKDsr4T8WNNBY6S
ENFgqxJddSytKUcOJxisxzRIaDs6TmA8hfkCLBRWslErOSObZwIoU0Zgxb6JEFmT5b0JbFDTIV99
AEdTYswM9QAqI4SnEJGWnRS7Ew9XZJxwqEThWAv5UMW97yW6tFXEuRXLYxXYxCE12fgczfwzzCAM
iy9hdS+b2XJmMhfxPtLblYgXh2CG2zjLInEQrWqqHVWdLCrEc+WTQ3blaIgJowlD4qhcrZPtb7pV
XCyxYgm6vvSLpVsk+OjWGsposTHjc/vOn7yGrQXMMuid2kb2VOg6uyWLImkfVuFx+75iNy6ZGZh8
1NJibXSOWxcz6TpXClzh/n7BAv16lddamW04p2sjCOypb7Ih86jnXFR004RNi7e+07fBq4PgpHAz
TvqlwDrfxr/OAElbnBUXkRu8lLISH9trik9HHOKZnD1xWoFUEFdLMJ96UtFSqRqLsj8yNcceXbuU
A5AsFuitWjLOhv0XkvQI3TFuhLW2IItVNSRqEQRqlz0s+dxDWAGYGv74UaNacP+Ijai9No2GuQNB
Igz+B7qhQTdETpMYenCe7gxFsMtTjHW4XsD7zz6FW2mNzrYS83iUHiI/B0FPa4gv49TTCmgNk4ca
k5ZFGZiMiCSiGGQJsoRCASrLLYzMjMJCioYi/fuLcyc4Xk9lhOXDBtrPEK/xuQs6MNAtmpXAAquT
DU4EXBBtsOUZgoMrLsw3Rw/KRaqKSELim7sinf3xz5sziXypbo40e8uPOabD+ya1Xnm6INWPfOoS
iOr/bWpYCWADiwY6veI8RcwUZvyW6rHuptGKwX+T7L1Yj1DdixTymNXrSkaSOGKPcmKCiOxFPFfr
99Od8KoK64zWrVIPDtikDTGJ2lCwn3f8R+LcQWG5j6oSIZgAVXjooJycqQpciZOuX3y6JwJhnuPh
8lx5yxfBXzmHDWNM6OGNOnySEFN8tIzQKD/AIcyWOjfziItfFKSLXPpnuBzBfot3KEgQcT4ODL+k
+GpX+N+eryWuqmnNqH/IO4L5JCAS0VJBGCoxAsjNEiXjkdWOP2XE9IJiRO4ffeExs7b3Q/iGLx10
Rew/dCYzo2j9gW/JR/1Zbx5ogeKsWUJTGgNFD8zSOvx8+1TH24alNEC/1RlCq6JvLCxReTZL/m2n
C9eSOLuH7wy5acFm9zT3XWzlsdNU1VrxBvRqBVDYojJgEHt2BvuQch4z47nuNCMFqwwuicrzpztg
oFzvJyJxPRdD9XT3+8NEILIR4e28jHe4GfDPw4vtGZutEvVMcAH7n5OJfmG4jwEVYjV798MQv1cp
2unyE6Dy/D9nGoqcHfi9xHukvQsxNCHmiN3ZPdouqVjs0c5o02jdL0f4mfphUDs947ItSTDaoAVC
2TrRMyaS3xA+rIp6sDLOgdrdN55e26TBn9jbySN5/WXhgAGaoZB+RHUsUOusNtYVzPUq6+gFAE+w
9S2/hBh0ipeneQwRHTM1C

In [25]:
xT = ddp.xs[-1]
q = xT[0:2]
v = xT[2:]
pin.forwardKinematics(robot_model, data, q, v)
pin.updateFramePlacements(robot_model, data)
print(data.oMf[robot_model.getFrameId("FOOT")])

  R =
  0.44463 -0.895714         0
 0.895714   0.44463         0
        0         0         1
  p = 0.0627921  -0.99802         0



In [ ]:
%matplotlib inline
# # Plotting the solution and the DDP convergence
crocoddyl.plotOCSolution(log.xs, log.us)
crocoddyl.plotConvergence(log.costs, log.u_regs, log.x_regs, log.grads, log.stops, log.steps)

# Visualizing the solution in gepetto-viewer
display.displayFromSolver(ddp)